In [37]:
import son_main_script as soon
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pymoo.indicators.hv import Hypervolume
import json
import os
import plotly.graph_objects as go
import numpy as np

In [38]:
# load dataset

from cProfile import label
from re import X
from matplotlib import axis

from son_pymoo import ObjectiveEnum


def sign_objective(value: float, key: str):
    if(key == ObjectiveEnum.AVG_DL_RATE.name or 
       key == ObjectiveEnum.AVG_RSSI.name or
       key == ObjectiveEnum.AVG_SINR.name or
       key == ObjectiveEnum.ENERGY_EFFICIENCY.name):
        return -value
    else:
        return value

network_name = "delete3"
param_configuration_list =["algorithm_config_3_STATIC"]
results_objectives_list = []
results_objective_spaces_list = []
result_dict_object = {"optimization_objectives": []}
ref_points = []
histories = []
for configuration_name_1 in param_configuration_list:
    results_directory_1 = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name_1)

    ## read json file 
    with open(results_directory_1 + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
        result_dict_object = json.load(openfile)
        ref_points = [1.1 for  x in result_dict_object["optimization_objectives"]]

        results_objectives_list.append(np.array(
        [[sign_objective(y[1][key], key) for key in result_dict_object["optimization_objectives"]]
        for y in [item for item in result_dict_object["results"]]]))
        histories.append({"name": configuration_name_1, "history": result_dict_object["history"]})
        
        results_objective_spaces_list.append(np.array(result_dict_object["objectiveSpace"]))

## prepare diagrams
fig2 = go.Figure()
fig2.update_layout(
    title="pareto fronts",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig1 = go.Figure()
fig1.update_layout(
    title="objective results",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig3 = go.Figure()
fig3.update_layout(
    title="Convergence",
    xaxis_title="Function Evaluations",
    yaxis_title="Hypervolume",
    legend_title="configuration names"
)
## normalize each solution and calculate hyper volume and add dataj to diagrams
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objectives_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)

for index, solution in enumerate(results_objectives_list):
    normalized_solution = np.array((solution - approx_ideal) / (approx_nadir - approx_ideal))

    fig2.add_trace(go.Scatter(x=normalized_solution[:,0], y=normalized_solution[:,1],
                    mode='markers',
                    name=param_configuration_list[index]))
    
    fig1.add_trace(go.Scatter(x=solution[:,0], y=solution[:,1],
                mode='markers',
                name=param_configuration_list[index] ))

## add data to convergence diagram
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objective_spaces_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)


for _, history  in enumerate(histories):
    hv_list_1 = [hv(np.array(solution)) for solution in history["history"]["objective_space_opt"]]
    
    fig3.add_trace(
        go.Scatter(
            x=history["history"]["n_evals"],
            y=hv_list_1,
            mode='lines+markers',
            name=history["name"]
        )
    )

fig2.show()
fig1.show()
fig3.show()



In [39]:

network_name = "delete3"
configuration_name_1 ="algorithm_config_7_LIVE"
configuration_name_2 ="algorithm_config_8_LIVE_greedy"
results_directory_1 = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name_1)
results_directory_2 = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name_2)
objective_np_array_1 = np.array([])
objective_np_array_1_negative = np.array([])
objective_np_array_2 = np.array([])
objective_np_array_2_negative = np.array([])
with open(results_directory_1 + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
    result_dict_object = json.load(openfile)
    objective_np_array_1 = np.array(result_dict_object)
    objective_np_array_1_negative = np.array(result_dict_object)
    objective_np_array_1_negative[:, 1:] *= -1
with open(results_directory_2 + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
    result_dict_object = json.load(openfile)
    objective_np_array_2 = np.array(result_dict_object)
    objective_np_array_2_negative = np.array(result_dict_object)
    objective_np_array_2_negative[:, 1:] *= -1



approx_ideal = np.concatenate((objective_np_array_1_negative[:, 1:], objective_np_array_2_negative[:, 1:]), axis=0).min(axis=0)
approx_nadir  = np.concatenate((objective_np_array_1_negative[:, 1:], objective_np_array_2_negative[:, 1:]), axis=0).max(axis=0)

normalized_objective_list_1 = np.array((objective_np_array_1_negative[:, 1:] - approx_ideal) / (approx_nadir - approx_ideal))
normalized_objective_list_2 = np.array((objective_np_array_2_negative[:, 1:] - approx_ideal) / (approx_nadir - approx_ideal))

hv = Hypervolume(ref_point=np.array([1.1, 1.1]),
                     norm_ref_point=False,
                     zero_to_one=False,
                     ideal=approx_ideal,
                     nadir=approx_nadir)

hv_list_1 = [hv(solution) for solution in normalized_objective_list_1]
hv_list_2 = [hv(solution) for solution in normalized_objective_list_2]


fig4 = go.Figure()

fig4.update_layout(
    title="hypervolume",
    xaxis_title="time in s",
    yaxis_title="Hypervolume",
    legend_title="dataset")

fig4.add_trace(
        go.Scatter(
            x=objective_np_array_1[:, 0],
            y=hv_list_1,
            mode='lines+markers',
            name=configuration_name_1
        ))

fig4.add_trace(
        go.Scatter(
            x=objective_np_array_1[:, 0],
            y=hv_list_2,
            mode='lines+markers',
            name=configuration_name_2
        ))


approx_ideal = np.concatenate((objective_np_array_1[:, 1:], objective_np_array_2[:, 1:]), axis=0).min(axis=0)
approx_nadir  = np.concatenate((objective_np_array_1[:, 1:], objective_np_array_2[:, 1:]), axis=0).max(axis=0)

normalized_objective_list_1 = np.array((objective_np_array_1[:, 1:] - approx_ideal) / (approx_nadir - approx_ideal))
normalized_objective_list_2 = np.array((objective_np_array_2[:, 1:] - approx_ideal) / (approx_nadir - approx_ideal))

fig6 = go.Figure()

fig6.update_layout(
    title= "normalized objectives",
    xaxis_title="time in s",
    yaxis_title="normalized objectives",
    legend_title="dataset")

fig6.add_trace(
        go.Scatter(
            x=objective_np_array_1[:, 0],
            y=normalized_objective_list_1[:, 0],
            name="energy_efficiency" + "_" +configuration_name_1,
            mode='lines+markers',
        ))
fig6.add_trace(
        go.Scatter(
            x=objective_np_array_1[:, 0],
            y=normalized_objective_list_1[:, 1],
            name="avrg_dl_datarate" + "_"+ configuration_name_1,
            mode='lines+markers',
        ))
fig6.add_trace(
        go.Scatter(
            x=objective_np_array_2[:, 0],
            y=normalized_objective_list_2[:, 0],
            name="energy_efficiency" + "_" + configuration_name_2,
            mode='lines+markers',
        ))
fig6.add_trace(
        go.Scatter(
            x=objective_np_array_2[:, 0],
            y=normalized_objective_list_2[:, 1],
            name="avrg_dl_datarate" + "_" + configuration_name_2,
            mode='lines+markers',
        ))

fig4.show()
fig6.show()